In [ ]:
# Create conda environment with necessary packages
# conda env create -f ERMA.yml (may take a while, use mamba if possible)
# set ipynb kernel to ERMA
# execute cells in order


# Move test file to execute folder
!cp data/test_epic_data.fastq.gz ../data/fastq/
# Start snakemake pipeline measuring process time
!time snakemake --cores all --use-conda --snakefile ../workflow/Snakefile --directory ../ 

<div>
<img src= "ERMA_workflow.png" width="1000"/>
</div>

In [ ]:
# 1. Step similarity search
# Input: fasta files
# Output: tabular similarity search results
# commands:
# usearch vs silva_v138.2 database (778 MB unzipped): usearch -usearch_local {input.fasta} -db {input.silva} -blast6out {output.silva_results} -evalue 1e-5 -threads {params.internal_threads} -strand plus -mincols 200 2> {log}
# blastx vs card_v3.3.0 database (4.8 MB unzipped): blastx -query {input.fasta} -db {params.db}/card_db -out {output.card_results} -outfmt 6 -evalue 1e-5 -num_threads {params.internal_threads} 2> {log}

import os,pathlib,pandas as pd
from IPython.display import display, Markdown

path = os.path.dirname(pathlib.Path().resolve())

silva_result = os.path.join(path,"results","test_epic_data","001","SILVA_results.txt.gz")
card_result = os.path.join(path,"results","test_epic_data","001","card_results.txt.gz")
string1 = "blastx -query {input.fasta} -db {params.db}/card_db -out {output.card_results} -outfmt 6 -evalue 1e-5 -num_threads {params.internal_threads} 2> {log}"
string2 = "usearch -usearch_local {input.fasta} -db {input.silva} -blast6out {output.silva_results} -evalue 1e-5 -threads {params.internal_threads} -strand plus -mincols 200 2> {log}"

blast_columns = ["query_id", "subject_id", "perc_identity", "align_length", "mismatches","gap_opens", "q_start", "q_end", "s_start", "s_end", "evalue", "bit_score"]


for i in [[card_result,"Card Similarity Search Result (BLASTX)",string1],[silva_result,"Silva Similarity Search Result (usearch)",string2]]:
    df = pd.read_csv(i[0],names=blast_columns,compression='gzip',sep='\t')
    df = df[df['evalue'] != 0]
    summary = df.agg({"perc_identity": ["mean", "min", "max"],"align_length": ["mean", "min", "max"],"evalue": ["mean", "min", "max"]}).style.format({"perc_identity": "{:.2f}","align_length": "{:.2f}","evalue": "{:.2e}"})
    display(Markdown(f"## {i[1]}"))
    display(Markdown(f" ```{i[2]}```"))
    display(Markdown(f" Shape: {df.shape}"))
    display(summary)    
    display(df.head(2))
    display(Markdown("---"))

In [ ]:
# 2. Integrate similarity search results
# Selfwritten python script "integrate_blast_data.py"
# Input: blastx, usearch results, ARO Mapping file
# Output: Processed integrated search results

aro_mapping = os.path.join(path, "data", "card_db", "aro_index.tsv")
aro_df = pd.read_csv(aro_mapping, sep="\t")

int_result = os.path.join(path, "results", "test_epic_data", "001","integrated_filtered_results.csv")
int_df = pd.read_csv(int_result, sep=",", low_memory=False)
int_df = int_df[int_df['evalue'] != 0]

display(Markdown(f"""
# Adding information to similarity results and integrate
---
## Mapping input: ARO File
- Shape: {aro_df.shape}\n
- Mapping to CARD results: ```df.merge(aro_df, on='ARO Accession', how='left')```
---
"""))

display(aro_df[aro_df["AMR Gene Family"] == "OXA beta-lactamase;OXA-48-like beta-lactamase"].head(2))

rows_16s = len(int_df[int_df["part"] == '16S'])
rows_abr = len(int_df[int_df["part"] == 'ABR'])
orientation_counts = int_df['orientation'].value_counts().to_dict()
subject_id_16 = int_df[int_df["part"] == "16S"]["subject_id"].iloc[0]
subject_id_abr = int_df[int_df["part"] == "ABR"]["subject_id"].iloc[0]

display(Markdown(f"""
## Additional Information Added
---
### 1. Adding field for discriminating 16S and ABR
Number of rows containing 16S information: **{rows_16s}**\n
Number of rows containing ABR information: **{rows_abr}**

### 2. Adding column for relevant information of similarity search "subject id"
genus = **{subject_id_16.split(";")[-2]}** <--- {subject_id_16}\n
ARO Accession = **{subject_id_abr.split("|")[-2]}** <--- {subject_id_abr}
### 3. Merge 16S and ABR results to one table (integrated_results.csv)
```combined_df = pd.concat([silva_df,card_df])```\n
"""))

part_16S = int_df[int_df["part"] == '16S'].head(1)
part_ABR = int_df[int_df["part"] == 'ABR'].head(1)
summary = int_df.agg({"perc_identity": ["mean", "min", "max"],"align_length": ["mean", "min", "max"],"evalue": ["mean", "min", "max"]}).style.format({"perc_identity": "{:.2f}","align_length": "{:.0f}","evalue": "{:.2e}"})

display(Markdown(f"""
## Integrated Results
- **Shape**: {int_df.shape}
---
### Summary Statistics
"""))

display(summary)
merged_df = pd.concat([part_16S, part_ABR])
columns_of_interest = ['query_id', 'subject_id', 'perc_identity', 'align_length','evalue','bit_score', 'part', 'primaryAccession','genus', 'ARO Accession','AMR Gene Family', 'Drug Class', 'Resistance Mechanism','CARD Short Name']
display(merged_df[columns_of_interest])

In [ ]:
# 3. Filter Blast results
# Selfwritten python script "filter_blast_results.py"
# Input: integrated_filtered_results.csv
# Output: filtered_results.csv

def process_orientation_and_counts(group):
    max_perc_identity = group["perc_identity"].max()
    filtered_group = group[group["perc_identity"] == max_perc_identity]
    return filtered_group

minsim = int_df[int_df['perc_identity'] > 80.0]

display(Markdown(f"""
# Filtering Steps
---
### 1. Filter by minimal similarity (default 0.8)\n
- ```f_df = df[df['perc_identity'] > 80.0]```\n
- Integrated dataframe before reduction: {int_df.shape[0]} vs after reduction **{minsim.shape[0]}** (reduction of {int_df.shape[0]-minsim.shape[0]})
---
"""))

abr_data = minsim[(minsim['part'] == 'ABR')]
abr_before = abr_data.shape[0]
abr_data = abr_data.groupby("query_id").apply(process_orientation_and_counts).reset_index(drop=True)
abr_after = abr_data.shape[0]

sixteen_s_data = minsim[(minsim['part'] == '16S')]
sixteen_s_before = sixteen_s_data.shape[0]
sixteen_s_data = sixteen_s_data.copy()
sixteen_s_data["query_id"] = sixteen_s_data["query_id"].str.split(expand=True)[0]
sixteen_s_data = sixteen_s_data.groupby("query_id").apply(process_orientation_and_counts).reset_index(drop=True)
sixteen_s_after = sixteen_s_data.shape[0]

display(Markdown(f"""
### 2. Keep maximum percentage hit per query and datapart (16S/ABR):\n
- Dataframe is split in dataparts (ABR and 16S): ```abr = f_df[f_df['part'] == ABR]```\n
- Dataparts are grouped by query_id and processed: ```f_df_max = abr.groupby('query_id').apply(give_max_hit)```\n
Datapart \'ABR\' before reduction: {abr_before} vs after reduction: **{abr_after}** (reduction of {abr_before-abr_after})\n
Datapart \'16S\' before reduction: {sixteen_s_before} vs after reduction: **{sixteen_s_after}** (reduction of {sixteen_s_before-sixteen_s_after})\n
---
"""))

common_query_ids = pd.Index(abr_data['query_id']).intersection(sixteen_s_data['query_id'])
abr_data_filtered = abr_data[abr_data['query_id'].isin(common_query_ids)]
sixteen_s_data_filtered = sixteen_s_data[sixteen_s_data['query_id'].isin(common_query_ids)]
remerged_df = pd.concat([abr_data_filtered, sixteen_s_data_filtered])

display(Markdown(f"""
### Step 3: Keep only intersectional hits
- Common query IDs found between ABR and 16S.
- Filtered data merged into a final dataset:\n
  ```common_hits = pd.Index(abr_data['query_id']).intersection(sixteen_s_data['query_id'])```\n
  ```abr_common = abr_data[abr_data['query_id'].isin(common)]```\n
  ```sixteen_s_common = sixteen_s_data[sixteen_s_data['query_id'].isin(common)]```\n
Datapart \'ABR\' before reduction: {abr_after} vs after reduction: **{abr_data_filtered.shape[0]}** (reduction of {abr_after-abr_data_filtered.shape[0]})\n
Datapart \'16S\' before reduction: {sixteen_s_after} vs after reduction: **{sixteen_s_data_filtered.shape[0]}** (reduction of {sixteen_s_after-sixteen_s_data_filtered.shape[0]})\n
---
"""))

part_ABR = abr_data_filtered.head(1)
part_16S = sixteen_s_data_filtered.head(1)
summary = remerged_df.agg({"perc_identity": ["mean", "min", "max"],"align_length": ["mean", "min", "max"],"evalue": ["mean", "min", "max"]}).style.format({"perc_identity": "{:.2f}","align_length": "{:.0f}","evalue": "{:.2e}"})

display(Markdown(f"""
### Final step: remerge filtered dataparts ABR/16S\n
```final_df = pd.concat([abr_common, sixteen_s_common])```\n
Filtered results before reduction: {int_df.shape[0]} vs after reduction: **{remerged_df.shape[0]}** (reduction of {int_df.shape[0]-remerged_df.shape[0]})\n
---
### -> filtered_result.csv\n
"""))

display(summary)
display(pd.concat([part_16S, part_ABR]))

In [ ]:
# General statistics of filtered vs integrated intermediate results (BLASTX and usearch)

display(Markdown(f"""
# Statistics of all integrated and filtered_files from Rozman (blastn and usearch)
- 12 Fasta files with 10 splits each
- similarity search blastx/blastn and blastx/usearch
data gathered with a bash for-loop:\n
```for i in *L001; do for j in $i/0*; do echo -ne "$j,$(cat $j/filtered_results.csv|wc -l),$(du -hs $j/filtered_results.csv)\\n" >> filtered_results_size.txt;done;done```\n
```for i in *L001; do for j in $i/0*; do echo -ne "$j,$(cat $j/integrated_filtered_results.csv|wc -l),$(du -hs $j/integrated_filtered_results.csv)\\n" >> integrated_filtered_results_size.txt;done;done```\n
plus 10 seconds manual change in the file preparing it as pandas input
"""))

# Define paths
filtered_path = os.path.join(path, "results_rozman", "filtered_results_size.txt")
integrated_path = os.path.join(path, "results_rozman", "integrated_filtered_results_size.txt")
filtered_path_us = os.path.join(path, "results_rozman", "filtered_results_size_rozman10-17_usearch.txt")
integrated_path_us = os.path.join(path, "results_rozman", "integrated_filtered_results_size_rozman10-17_usearch.txt")

# Function to convert sizes to GB
def convert_size_to_gb(size):
    if size.endswith("K"):
        return float(size.replace("K", "")) / 1000000  # Convert MB to GB
    elif size.endswith("M"):
        return float(size.replace("M", "")) / 1000  # Convert MB to GB
    elif size.endswith("G"):
        return float(size.replace("G", ""))  # Already in GB
    else:
        raise ValueError(f"Unexpected size format: {size}")

# Function to process a single file
def process_file(file_path):
    df = pd.read_csv(file_path, sep=",", header=0)
    df["lines"] = df["lines"].replace(",", "", regex=True).astype(int)  # Ensure numeric
    df["size_GB"] = df["size"].apply(convert_size_to_gb)
    summary_per_file = df.groupby("fasta-file").agg(
        total_lines=("lines", "sum"),
        total_size_GB=("size_GB", "sum"),
        average_lines=("lines", "mean"),
        average_size_GB=("size_GB", "mean"),
    )
    overall_summary = {
        "Total Lines (All Files)": df["lines"].sum(),
        "Total Size (All Files) (GB)": df["size_GB"].sum(),
        "Average Lines per Fasta File": summary_per_file["total_lines"].mean(),
        "Average Size per Fasta File (GB)": summary_per_file["total_size_GB"].mean(),
    }
    return summary_per_file, overall_summary

# Process both files
filtered_summary, filtered_overall = process_file(filtered_path)
integrated_summary, integrated_overall = process_file(integrated_path)
filtered_summary_us, filtered_overall_us = process_file(filtered_path_us)
integrated_summary_us, integrated_overall_us = process_file(integrated_path_us)

# Combine summaries for comparison
combined_summary = pd.concat(
    [filtered_summary, integrated_summary],
    axis=1,
    keys=["Filtered", "Integrated"]
)

combined_summary2 = pd.concat(
    [filtered_summary_us, integrated_summary_us],
    axis=1,
    keys=["Filtered_usearch", "Integrated_usearch"]
)

# Format combined summary
for col in combined_summary.columns:  # Iterate over all columns
    if "lines" in col[1]:
        combined_summary[col] = combined_summary[col].map('{:,.0f}'.format)
    elif "size_GB" in col[1]:
        combined_summary[col] = combined_summary[col].map('{:.2f}'.format)
    else:
        print(col,"not found")

for col in combined_summary2.columns:  # Iterate over all columns
    if "lines" in col[1]:
        combined_summary2[col] = combined_summary2[col].map('{:,.0f}'.format)
    elif "size_GB" in col[1]:
        combined_summary2[col] = combined_summary2[col].map('{:.4f}'.format)
    else:
        print(col,"not found")


# Combine overall summaries for comparison
combined_overall = {
    "Filtered": filtered_overall,
    "Integrated": integrated_overall,
}

combined_overall2 = {
    "Filtered": filtered_overall_us,
    "Integrated": integrated_overall_us,
}

# Display results
print("Combined Summary per Fasta File witch blastn:")
display(combined_summary)

print("Combined Summary per Fasta File with usearch:")
display(combined_summary2)

print("\nOverall Summary Comparison with blastn:")
for key in combined_overall["Filtered"]:
    print(f"Filtered:\t{key}:\t{combined_overall['Filtered'][key]:,.2f}")
    print(f"Integrated:\t{key}:\t{combined_overall['Integrated'][key]:,.2f}")

print("\nOverall Summary Comparison with usearch:")
for key in combined_overall2["Filtered"]:
    print(f"Filtered:\t{key}:\t{combined_overall2['Filtered'][key]:,.2f}")
    print(f"Integrated:\t{key}:\t{combined_overall2['Integrated'][key]:,.2f}")    


In [ ]:
# 4. Create abundance table and plot abundance
# Selfwritten python script "generate_genus_distribution_plot.py"
# Input: all filtered_result.csv parts of one sample
# Output: abundance plot over all ABRs

import altair as alt

input_files = ["/local/work/adrian/ERMA/results/test_epic_data/001/filtered_results.csv"]
necessary_columns = ["query_id","part","genus","AMR Gene Family"]
sample_name = "test_epic_data"

df = pd.read_csv(input_files[0], sep=",", header=0)
first_query = df["query_id"].iloc[1]
test = df[df["query_id"] == first_query]

display(Markdown(f"""
## Create abundance table
---
### 1. Parse filtered results with reduced columns, drop duplicates and merge (see cell before)
### 2. Merge split fasta parts in one dataframe\n
### 3. Groupy by AMR Gene Family and genus (export table)\n
```combined_df.groupby(['AMR Gene Family_abr', 'genus_16S']).size()```\n
### 4. Plot count per AMR Gene Family
---
## Create abundance plot
"""))

all_data = []
for input_file in input_files:
        df = pd.read_csv(input_file, sep=",", usecols=necessary_columns, header=0)
        df = df.drop_duplicates()
        # Split and merge ABR and 16S data by query_id
        abr = df[df["part"] == "ABR"]
        sixteen_s = df[df["part"] == "16S"]
        df_merged = pd.merge(abr, sixteen_s, on='query_id', suffixes=('_abr', '_16S'))

        # Assign genus based on 16S path for each merged record
        all_data.append(df_merged)  # Append the DataFrame to the list
        # Concatenate all DataFrames into one
combined_df = pd.concat(all_data, ignore_index=True)

# Group by AMR gene family and genus across all files
genus_distribution = combined_df.groupby(['AMR Gene Family_abr', 'genus_16S']).size().reset_index(name='count')
display(genus_distribution)
# Create the Altair bar chart
chart = alt.Chart(genus_distribution).mark_bar().encode(
        x=alt.X('AMR Gene Family_abr', title='AMR Gene Family'),
        y=alt.Y('count', title='Count'),
        color=alt.Color('genus_16S', title='Genus'),
        tooltip=['AMR Gene Family_abr', 'genus_16S', 'count']
).properties(
title=f'Genus Distribution for {sample_name}',
width=800,
height=400
)

# Save the plot as an HTML file
chart

In [ ]:
import plotly.express as px

# 5. Create bubble plot
# Selfwritten python script "generate_genus_distribution_plot.py"
# Input: abundance file
# Output: bubble plot per sample

display(Markdown(f"""
# Create bubble plot
---
### 1. Parse abundance file
### 2. Groupy by AMR Gene Family and genus (export table)\n
```combined_df.groupby(['AMR Gene Family_abr', 'genus_16S']).size()```\n
### 3. Reduce to ABR with most total hits
### 4. Plot sample vs genus with relative genus count as size and total genus count as colour              
---
"""))

def create_bubble_plots(df, abundance_threshold, output1,output2):
    # Iterate over unique AMR Gene Families
    # Filter data for the current AMR Gene Family
    df = pd.read_csv(df,header=0,sep=',')
    total_counts_per_abr = df.groupby('AMR Gene Family')['genus_count'].sum()
    top_abr = total_counts_per_abr.idxmax()
    top_abr_data = df[df['AMR Gene Family'] == top_abr]
    top_abr_data = top_abr_data[top_abr_data['relative_genus_count'] > float(abundance_threshold)]
    # Create the bubble plot
    fig = px.scatter(
        top_abr_data, 
        x="sample", 
        y="genus", 
        size="relative_genus_count",
        color="total_genus_count", 
        hover_name="genus",
        hover_data={
            "genus_count": True,
            "relative_genus_count": True,
            "total_genus_count": True,
            "sample": False
        },
        size_max=20,
        color_continuous_scale="Greens"
    )
    
    # Update layout for titles and axis labels
    fig.update_layout(
        title=f'Bubble Plot of Relative Genera Abundance per Sample and top found AMR:<br> {top_abr} with {total_counts_per_abr.sum()} fusion reads over all samples',
        xaxis_title='Sample - AMR Gene Family',
        yaxis_title='Genus',
        coloraxis_colorbar=dict(title="Total Genus Count"),
        #paper_bgcolor='grey',
        plot_bgcolor='lightgrey',
        yaxis=dict(categoryorder="category descending")
    )
    fig.show()

if __name__ == "__main__":
    input_files = "/local/work/adrian/ERMA/results/abundance/combined_genus_abundance.csv"
    output_html = "snakemake.output[0]"
    output_csv = "snakemake.output[1]"
    abundance_threshold = 0.001
    create_bubble_plots(input_files, abundance_threshold, output_html,output_csv)

In [ ]:
# 6. Create boxplots
# Selfwritten python script "percidt_per_genus.py"
# Input: all filtered_result.csv parts of one sample
# Output: boxplot over all samples per percentage identity, number of unique hits and genera

import matplotlib.pyplot as plt
import seaborn as sns

display(Markdown(f"""
# Create boxplot
---
### 1. Parse filtered results with reduced columns, drop duplicates and merge (see cell before)
### 2. Merge split fasta parts in one dataframe\n
### 3. Groupy by AMR Gene Family and genus (export table)\n
```combined_df.groupby(['AMR Gene Family_abr', 'genus_16S']).size()```\n
### 4. Plot count per AMR Gene Family
---
## Plot abundance
"""))

def generate_percentage_idt_per_genus(input_files, output_file):
        
    # Combine all partitions into a single DataFrame
    combined_data = pd.read_csv(input_files, sep=",", header=0)
    
    # Calculate genus query counts and genus order
    genus_query_counts = combined_data.groupby('genus')['query_id'].nunique().reset_index()
    genus_query_counts.columns = ['genus', 'unique_query_count']
    combined_data = pd.merge(combined_data, genus_query_counts, on='genus')
    genus_order = genus_query_counts.sort_values(by='unique_query_count', ascending=False)['genus']
    
    # Plotting
    fig, ax1 = plt.subplots(figsize=(15, 8))
    sns.boxplot(x='genus', y='perc_identity', data=combined_data, ax=ax1, order=genus_order,fliersize=0.0, color='dodgerblue')
    ax1.set_xlabel("Bacterial Genus")
    ax1.set_ylabel("Percentage Identity", color='royalblue')
    ax1.set_title("Boxplot of Percentage Identity and Read Counts for Each Bacterial genus")
    ax1.set_xticklabels(ax1.get_xticklabels(), rotation=90)

    # Add a second y-axis for unique query counts
    ax2 = ax1.twinx()
    sns.barplot(x='genus', y='unique_query_count', data=genus_query_counts, ax=ax2, alpha=0.3, color='purple', order=genus_order)
    ax2.set_ylabel("Number of Unique Reads (query_id)", color='violet')
    
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    input_files = "/local/work/adrian/ERMA/results/test_epic_data/001/filtered_results.csv"
    output_file = "snakemake.output[0]"
    generate_percentage_idt_per_genus(input_files, output_file)
